In [151]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 15 17:17:30 2019

@author: viryl
"""


from ipdb import set_trace

import os
#os.chdir(' ')   #设置当前路径
import scipy.io   
import scipy.ndimage
import numpy as np
from random import shuffle   #随机排序
from Utils import pca, pad, standartize, patch


# 定义全局变量
#PATCH_SIZE = 5  # 切片尺寸
#PATCH_SIZE = 7 # 切片尺寸
#PATCH_SIZE = 9 # 切片尺寸
#PATCH_SIZE = 11# 切片尺寸
#PATCH_SIZE = 13# 切片尺寸
#PATCH_SIZE = 15# 切片尺寸
#PATCH_SIZE = 17# 切片尺寸
PATCH_SIZE = 19# 切片尺寸
# PATCH_SIZE = 21# 切片尺寸
OUTPUT_CLASSES = 3  # 输出9类地物   #已改   ，OUTPUT_CLASSES = 9
TEST_FRAC = 0.50  # 用来测试数据的百分比

NEW_DATA_PATH = os.path.join('/home/liyuan/dataset/HSI/高光谱医学数据/00microscope medical image data/medical sample 1-3/', "patch")  # 存放数据路径 patch是文件夹名称


In [152]:


# 加载数据
def loadData(fileName):
    # 原始数据路径
    data = scipy.io.loadmat(fileName)
    
    image = data['image']   #已改data = data[list(data.keys())[-1]] 
    label = data['map']
    
    image = np.transpose(image, (2, 0, 1))  # 将通道数提前，便于数组处理操作
    return image, label


# 生成切片数据并存储
def createdData(X, label):
    PATCH, LABEL = [], []
    for c in range(OUTPUT_CLASSES):
      
        for h in range(X.shape[1]-PATCH_SIZE+1):
            for w in range(X.shape[2]-PATCH_SIZE+1):
                
                gt = label[h, w]
                if(gt == c):
                    
                    img = patch(X, PATCH_SIZE, h, w)
                    PATCH.append(img)
                    LABEL.append(gt)
        
    return PATCH, LABEL
        
        




In [153]:
#data, label = loadData("PaviaU", "PaviaU.mat", "PaviaU_gt.mat")
#basePath = '/home/liyuan/dataset/HSI/高光谱医学数据/00microscope medical image data/'
basePath='/home/liyuan/dataset/HSI/高光谱医学数据/00microscope medical image data/medical sample 1-3/bloodcell1-3.mat'
data, label = loadData(basePath )
#data=data[:,100:580,100:480]   # 新加，取100*100个像素点
#label=label[100:580,100:480]   #新加


In [154]:
data = standartize(data)
data = pad(data, int((PATCH_SIZE-1)/2))
   #储存数据
PATCH, LABEL = createdData(data, label)
#    set_trace()
#         # 打乱切片
#         shuffle(PATCH)
#         # 划分测试集与训练集
#         split_size = int(len(PATCH)*TEST_FRAC)
#         TEST_PATCH.extend(PATCH[:split_size])  # 0 ~ split_size
#         TRAIN_PATCH.extend(PATCH[split_size:])  # split_size ~ len(class)
#         TEST_LABEL.extend(LABEL[:split_size])
#         TRAIN_LABEL.extend(LABEL[split_size:])

In [155]:
patch = np.array(PATCH)
label = np.array(LABEL)

In [156]:
patchsize_path = os.path.join(NEW_DATA_PATH, str(PATCH_SIZE))
if not os.path.exists(patchsize_path):
    os.makedirs(patchsize_path)
np.save(os.path.join(patchsize_path, 'GND.npy'), label)

In [157]:
patchsize_path = os.path.join(NEW_DATA_PATH, str(PATCH_SIZE))
if not os.path.exists(patchsize_path):
    os.makedirs(patchsize_path)
np.save(os.path.join(patchsize_path, 'PATCH.npy'), patch)